#### Taken from Global Nav

In [1]:
import time
import asyncio
from tdmclient import aw, ClientAsync
import math
import numpy as np

# conversion thymio speed to mm/s
Thymio_to_mms = 0.349
px_to_mm = 140/100
#Thymio_to_pxs = Thymio_to_mms * mm_to_px 

# Thymio connection
async def connect_Thymio():
    """
    Establish a connection with the Thymio if possible
    """
    global node, client
    try:
        client = ClientAsync()
        node = await asyncio.wait_for(client.wait_for_node(), timeout=2.0)
        await node.lock()
        print("Thymio connected")

    except asyncio.TimeoutError:
        print("Thymio not connected: Timeout while waiting for node.")
    except Exception as e:
        print(f"Thymio not connected: {str(e)}")
        
# Thymio disconnection
def disconnect_Thymio():
    """
    Enable to disconnect the Thymio
    """
    aw(node.stop())
    aw(node.unlock())
    print("Thymio disconnected")
        
# Thymio control motor speeds  
async def set_speeds(left_speed, right_speed):
    """
    Enable to set the speed of the Thymio's wheels
    """
    global node
    v = {
        "motor.left.target":  [left_speed],
        "motor.right.target": [right_speed],
    }
    await node.set_variables(v)
    
async def motors_stop():
    """
    Stop the Thymio
    """
    global node
    v = {
        "motor.left.target":  [0],
        "motor.right.target": [0],
    }
    await node.set_variables(v)    

# Turn a specified angle 

# Constants
ROTATION_SPEED = 100
TIME_FULL_TURN = (8800/1000)

async def turn(angle):
    # Calculate the time needed to turn through the required angle
    rotation_time = (abs(angle) / (2*np.pi)) * TIME_FULL_TURN

    # Turn robot on itself
    # Check the sign of angle
    if np.sign(angle) > 0:
        # If angle is positive, turn in one direction
        await set_speeds(ROTATION_SPEED, -ROTATION_SPEED)
    else:
        # If angle is negative, turn in the other direction
        await set_speeds(-ROTATION_SPEED, ROTATION_SPEED)

    # Wait required time
    time.sleep(rotation_time)

    # Stop the robot
    #await motors_stop()

# Constants
FORWARD_SPEED = 200  # Base speed
TIME_PER_MM = 15.5/1000  # Time it takes for the robot to travel one meter at base speed

async def move_forward(distance_px):
    # Calculate the time needed to travel the requested distance
    
    distance_mm = distance_px * px_to_mm
    travel_time = (distance_mm) * TIME_PER_MM
    
    # Robot moves forward
    await set_speeds(FORWARD_SPEED, FORWARD_SPEED)

    # Wait for the necessary time
    time.sleep(travel_time)
    #print("End of forward after : ", travel_time)

In [4]:
await connect_Thymio()

Thymio connected


In [5]:
disconnect_Thymio()

Thymio disconnected


# Local Navigation 

In [6]:
node = await client.wait_for_node()
await node.lock()

Node 7ceccd72-4650-4181-812c-177085571dc0

In [7]:
await node.wait_for_variables()
node.var

{'_fwversion': [14, 0],
 '_id': [-1654],
 '_productId': [8],
 'button.backward': [0],
 'button.center': [0],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._raw': [11464, 9859, 14850, 14794, 16167],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-1654],
 '_imot': [0, 0],
 '_vbat': [971, 972],
 'buttons._mean': [11453, 9842, 14866, 14777, 16148],
 'buttons._noise': [101, 73, 103, 90, 101],
 'motor.left.speed': [-5],
 'motor.left.target': [0],
 'motor.right.speed': [0],
 'motor.right.target': [0],
 'prox.comm.rx': [0],
 'prox.comm.rx._intensities': [0, 0, 0, 0, 0, 0, 0],
 'prox.comm.rx._payloads': [0, 0, 0, 0, 0, 0, 0],
 'prox.comm.tx': [0],
 'prox.ground.ambiant': [4, 3],
 'prox.ground.delta': [1014, 1013],
 'prox.ground.reflected': [884, 877],
 'prox.horizontal': [3040, 3136, 3080, 2504, 4751, 0, 0],
 '_integrato

In [8]:
# Connect to the Thymio robot
client.tdm_addr
client.tdm_port

8596

### Desciption
Local navigation allows modulating the trajectory to avoid unforeseen, local obstacles. It often pushes the controller to leave the optimal path to make an avoidance manoeuvre. Once the obstacle is passed, the controller can go back to the optimal path or find a new one. An important issue is to decide when the controller has to start avoiding, and, more difficult, when the obstacle can be considered as passed.

### Additional imports

In [9]:
# new
import serial

### Parameters 

In [10]:
## Parameters for local navigation
pi = math.pi

obstThrL = 10     # low obstacle threshold to switch state 1->0 (from global nav to local)
obstThrH = 20     # high obstacle threshold to switch state 0->1 (from local nav to global)
obstThrO = 2500   # threshold for determining obstacles

### Potential Field Navigation?

In [44]:
# 1 
async def pt_field():
    """
    - Obstacle detected right in front: only rotate
    - Obstacle detected near sides: rotate + go forward
    - Determine rotation left or right
    """
    # Read sensor values
    global node
    await node.wait_for_variables()
    # sensor_prox = node.v.prox.horizontal 
    for i in range(10):
        sensor_prox = node["prox.horizontal"]
        print(list(sensor_prox))
        
        # Call obstacle avoidance function
        await turn_move(sensor_prox) # Rotate left or right depending on sensor values
        await client.sleep(0.2)
    await motors_stop()

async def turn_move(sensor_prox):
    # Object detected right in front
    if sensor_prox[2] > (sensor_prox[1]+sensor_prox[3]):
        # Rotate only
        await turn(pi/2) # Rotate 90 degrees to the left
    
    # Left sensors weights > Right sensors
    # Turn right
    elif (sensor_prox[0]+sensor_prox[1]) > (sensor_prox[3]+sensor_prox[4]):
        await turn(pi/4)
        await move_forward(1) 
        # await turn('right',sensor_prox, threshold_loc,local_motor_speed)
        
    # Right sensors weights > Left sensors
    # Turn left    
    elif (sensor_prox[3]+sensor_prox[4]) > (sensor_prox[0]+sensor_prox[1]):
        await turn(-pi/4)
        await move_forward(1)
        # await turn('left',sensor_prox, threshold_loc,local_motor_speed)
   
    # Detect nothing, just move forward
    else:
        await move_forward(0.05)
        await motors_stop()

In [47]:
# 2 - Added local obstacle threshold
async def pt_field():
    """
    - Obstacle detected right in front: only rotate
    - Obstacle detected near sides: rotate + go forward
    - Determine rotation left or right
    - 
    """
    # Read sensor values
    global node
    await node.wait_for_variables()
    # sensor_prox = node.v.prox.horizontal 
    for i in range(10): # while True:
        sensor_prox = node["prox.horizontal"]
        print(list(sensor_prox))
        
        # Call obstacle avoidance function
        await turn_move(sensor_prox) # Rotate left or right depending on sensor values
        await client.sleep(0.2)
    await motors_stop()

async def turn_move(sensor_prox):
    # Object detected right in front
    if sensor_prox[2] > (sensor_prox[0]+sensor_prox[1]+sensor_prox[3]+sensor_prox[4]):
        # Rotate only
        await turn(pi/2) # Rotate 90 degrees to the left
    else:
        # Left sensors weights > Right sensors
        # Turn right
        if (sensor_prox[0]+sensor_prox[1]) > (sensor_prox[3]+sensor_prox[4]):
            while sum(sensor_prox[i] > obstThrO for i in range(0,5)) > 0:
                await turn(pi/6) 
            while sensor_prox[0] > obstThrH:
                await move_forward(1)
            # turn back?
            await turn(-pi/6)
        # Right sensors weights > Left sensors
        # Turn left    
        elif (sensor_prox[3]+sensor_prox[4]) > (sensor_prox[0]+sensor_prox[1]):
            while sum(sensor_prox[i] > obstThrO for i in range(0,5)) > 0:
                await turn(-pi/6)
            while sensor_prox[4] > obstThrH:
                await move_forward(1)
            # turn back?
            await turn(pi/6)
        # Detect nothing, just move forward
        else:
            await move_forward(0.05)
            await motors_stop()

In [48]:
# 3 - Wall following
async def pt_field():
    """
    - Obstacle detected right in front: only rotate
    - Obstacle detected near sides: rotate + go forward
    - Determine rotation left or right
    - Wall following of the obstacle
    """
    # Read sensor values
    global node
    await node.wait_for_variables()
    # sensor_prox = node.v.prox.horizontal 
    for i in range(10): # while True:
        sensor_prox = node["prox.horizontal"]
        print(list(sensor_prox))
        
        # Call obstacle avoidance function
        await turn_move(sensor_prox) # Rotate left or right depending on sensor values
        await client.sleep(0.2)
    await motors_stop()

async def turn_move(sensor_prox):
    # Object detected right in front
    if sensor_prox[2] > (sensor_prox[0]+sensor_prox[1]+sensor_prox[3]+sensor_prox[4]):
        # Rotate only
        await turn(pi/2) # Rotate 90 degrees to the left
    else:
        # Calculate difference in Left & Right sensors
        diff_lr = sensor_prox[0] - sensor_prox[4]
        
        # Wall-following
        if abs(diff_lr) < obstThrL: 
            await move_forward(1)
        elif diff_lr > 0: 
            await turn(pi/24)
            await move_forward(0.1)
        else: 
            await turn(-pi/24)
            await move_forward(0.1)
        
        # Detect nothing, just move forward
#         else:
#             await move_forward(0.05)
#             await motors_stop()

In [39]:
await connect_Thymio()

Thymio connected


In [52]:
# Test
await pt_field()

[0, 0, 3190, 4739, 5005, 0, 0]
[0, 0, 0, 3011, 4422, 0, 0]
[0, 0, 0, 3018, 4833, 0, 0]
[0, 0, 0, 0, 4922, 0, 0]
[0, 0, 0, 1382, 0, 0, 0]
[0, 0, 1668, 1874, 0, 0, 0]
[0, 0, 2694, 1992, 1718, 0, 0]
[0, 0, 5029, 2415, 3763, 0, 0]
[0, 0, 0, 3146, 4558, 0, 0]
[0, 0, 0, 3253, 4204, 0, 0]


In [53]:
disconnect_Thymio()

Thymio disconnected
